# Building a Recommendation System


## I would be trying these below methods for this dataset

1. Content Based Filtering
2. Collaberative Filtering
3. Hybrid Filtering

## Content Based Filtering 
Recommends a product similar to the product or based on recent activity

In [26]:
# Importing Packages
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [100]:
# Data Loading 
orders = pd.read_csv('orders_data.csv')


In [101]:
# Data Loading 
orders = orders[orders['eval_set']=='train']

In [102]:
orders=  orders.groupby('product_name', as_index=False).agg(set)

In [103]:
orders = orders.reset_index()

In [90]:
orders.head(5)

,index,product_name,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,aisle,department,aisle-department
0,0,#2 Coffee Filters,"{24208, 1340305, 704405, 2816283, 2767517, 209...","{54923, 63759, 156561, 123672, 188955, 104604,...",{train},"{33, 36, 4, 5, 7, 8, 6, 9, 11, 41, 12, 17, 18,...","{0, 1, 2, 3, 4, 5, 6}","{10, 11, 12, 13, 14, 15, 17, 18, 19, 21}","{4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 13.0, 25....",{25773.0},"{1.0, 2.0, 34.0, 4.0, 5.0, 6.0, 8.0, 9.0, 10.0...","{0.0, 1.0}",{26.0},{7.0},{coffee},{beverages},{coffee beverages}
1,1,#2 Cone White Coffee Filters,{3052036},{157701},{train},{10},{3},{10},{30.0},{2158.0},{15.0},{0.0},{26.0},{7.0},{coffee},{beverages},{coffee beverages}
2,2,#2 Mechanical Pencils,{2317343},{70051},{train},{6},{2},{16},{3.0},{26685.0},{5.0},{0.0},{87.0},{17.0},{morehousehold},{household},{morehousehold household}
3,3,#4 Natural Brown Coffee Filters,"{2858756, 3413995, 2638064, 2460790, 1763739, ...","{52930, 16290, 192458, 33327, 82582, 164760}",{train},"{100, 6, 40, 10, 13, 15}","{0, 2, 6}","{8, 9, 14, 17}","{3.0, 6.0, 8.0, 13.0, 16.0, 18.0}",{26381.0},"{3.0, 4.0, 5.0, 6.0, 8.0, 13.0}","{0.0, 1.0}",{26.0},{7.0},{coffee},{beverages},{coffee beverages}
4,4,& Go! Hazelnut Spread + Pretzel Sticks,"{1551457, 2937505, 3414341, 1172967, 986171, 4...","{7588, 78825, 104745, 117353, 130671, 32726, 3...",{train},"{4, 5, 6, 7, 8, 39, 30}","{1, 2, 4, 6}","{9, 10, 11, 12, 13, 16, 17, 22}","{6.0, 7.0, 8.0, 13.0, 14.0, 28.0, 30.0}",{20604.0},"{4.0, 5.0, 8.0, 9.0, 14.0, 17.0, 25.0, 28.0, 3...","{0.0, 1.0}",{88.0},{13.0},{spreads},{pantry},{spreads pantry}


In [71]:
len(orders), orders.columns

(39123,
 Index(['index', 'product_id', 'order_id', 'user_id', 'eval_set',
        'order_number', 'order_dow', 'order_hour_of_day',
        'days_since_prior_order', 'add_to_cart_order', 'reordered',
        'product_name', 'aisle_id', 'department_id', 'aisle', 'department',
        'aisle-department'],
       dtype='object'))

In [151]:
orders.product_name=orders.product_name.str.replace('[#,@,&,!,$,%,^,*,+,-,",.,\,/]','') # Removing Special symbol in product name column

orders.product_name = orders.product_name.str.replace('\d+', '') # remving numbers from the product name

orders.product_name = orders.product_name.str.lstrip() # Removing Leading white spaces in the product name

#### Cleaning the product name column

In [137]:
#orders.product_name = orders.product_name.str.replace('[#,@,&,!,$,%,^,*,+,-]','').str.replace('\d+', '').str.lstrip()

In [144]:
orders.head(3)

,index,product_name,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,aisle,department,aisle-department
0,0,Coffee Filters,"{24208, 1340305, 704405, 2816283, 2767517, 209...","{54923, 63759, 156561, 123672, 188955, 104604,...",{train},"{33, 36, 4, 5, 7, 8, 6, 9, 11, 41, 12, 17, 18,...","{0, 1, 2, 3, 4, 5, 6}","{10, 11, 12, 13, 14, 15, 17, 18, 19, 21}","{4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 13.0, 25....",{25773.0},"{1.0, 2.0, 34.0, 4.0, 5.0, 6.0, 8.0, 9.0, 10.0...","{0.0, 1.0}",{26.0},{7.0},{coffee},{beverages},coffee beverages
1,1,Cone White Coffee Filters,{3052036},{157701},{train},{10},{3},{10},{30.0},{2158.0},{15.0},{0.0},{26.0},{7.0},{coffee},{beverages},coffee beverages
2,2,Mechanical Pencils,{2317343},{70051},{train},{6},{2},{16},{3.0},{26685.0},{5.0},{0.0},{87.0},{17.0},{morehousehold},{household},morehousehold household


In [145]:
orders['aisle-department'] = orders.apply(lambda x : ' ' + ' '.join(x['aisle']) + ' ' + ' '.join(x['department']), axis = 1)


### Using Cosine Similarily

If the output value is 0, no similar and if it is closer to 1 then more similar

In [152]:
count_vec = CountVectorizer(stop_words='english')
count_vec_matrix = count_vec.fit_transform(orders['aisle-department'])
cosine_sim_matrix = cosine_similarity(count_vec_matrix, count_vec_matrix)
mapping = pd.Series(orders.index, index = orders['product_name'])

In [110]:
mapping

product_name
Coffee Filters                                                       0
Cone White Coffee Filters                                            1
Mechanical Pencils                                                   2
Natural Brown Coffee Filters                                         3
Go Hazelnut Spread  Pretzel Sticks                                   4
                                                                 ...  
with Xylitol Unwrapped Original Flavor  Sticks Sugar Free Gum    39118
with Xylitol Unwrapped Spearmint  Sticks Sugar Free Gum          39119
with Xylitol Watermelon Twist  Sticks Sugar Free Gum             39120
with a Splash of Mango Coconut Water                             39121
with a Splash of Pineapple Coconut Water                         39122
Length: 39123, dtype: int64

### Selecting a product

In [153]:
product_name = input("Please enter a product name :\n")

Please enter a product name :
 Go Hazelnut Spread  Pretzel Sticks


In [154]:
top_n_values = input("Enter the top n values to be listed")

Enter the top n values to be listed 10


In [155]:
def recommend_products(product_name):
    product_index = mapping[product_name]
    
    similarity_score = list(enumerate(cosine_sim_matrix[product_index]))
    similarity_score = sorted(similarity_score, key= lambda x:x[1], reverse=True)
    
    similarity_score = similarity_score[1:int(top_n_values)+1]
    product_indexes = [i[0] for i in similarity_score]
    
    return (orders['product_name'].iloc[product_indexes])

recommended_products =  pd.DataFrame(recommend_products(product_name))

In [156]:
print ('Below are the Top {} products of {} \n\n\n {}'.format(top_n_values,product_name,recommended_products['product_name']))


Below are the Top 10 products of Go Hazelnut Spread  Pretzel Sticks 


 82               All Natural Creamy Peanut Butter
83              All Natural Crunchy Peanut Butter
196                  Natural Creamy Peanut Butter
197         Natural Creamy Unsalted Peanut Butter
198                 Natural Crunchy Peanut Butter
207    Natural Old Fashioned Creamy Peanut Butter
253                    Organic Raw Coconut Butter
254     Organic Raw Pecan Nut Butter With Cashews
960       Less Fat Organic Powdered Peanut Butter
997                                A Red Duet Jam
Name: product_name, dtype: object


In [157]:
(recommended_products['product_name']).to_string(index=False).replace('\n',',')

'           All Natural Creamy Peanut Butter,          All Natural Crunchy Peanut Butter,               Natural Creamy Peanut Butter,      Natural Creamy Unsalted Peanut Butter,              Natural Crunchy Peanut Butter, Natural Old Fashioned Creamy Peanut Butter,                 Organic Raw Coconut Butter,  Organic Raw Pecan Nut Butter With Cashews,    Less Fat Organic Powdered Peanut Butter,                             A Red Duet Jam'

### Conclusion of Content based filtering

__Based on the similar products, aisle and department, for each given product, we are able to list the similar product based on cosine similarity score__